# Monte Carlo Project

## Iacob Jessica, Mourre Grégoire

In this Jupter file, we will implement the different methods presented and described in https://github.com/gregoiremrr/Monte-Carlo-for-American-Options.


In [79]:
import numpy as np
from numpy.random import default_rng

# Longstaff and Schwartz method for the lower bound

In [80]:
n = 5000
m = 10
rng = default_rng()

r, sigma, x, K, T = 0, .3, 100, 105, 1
dt = T/m

def g(x, K=K):
    return np.maximum(K-x,0)

def phi(x):
    return np.array([1, x, x**2, g(x), x**3, x**4, x**5], dtype=object)
l = len(phi(0))

def reg(x, *alphas):
    alpha = np.array(alphas)
    return np.sum(alpha * phi(x))

In [81]:
# Tsitsiklis & Van Roy's algorithm

# Step 1
B = rng.normal(size=m*n).reshape(m,n)
X = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B]), axis=0)

# Step 2
V = np.zeros(n*(m+1)).reshape(m+1,n)
V[-1,:] = g(X[-1,:])

# Step 3
from scipy.optimize import curve_fit
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    alpha_star = curve_fit(reg, X[i,:], V[i+1,:], p0=alpha0)[0]
    _1 = g(X[i,:])
    _2 = reg(X[i,:], alpha_star)
    V[i,:] = _1 * (_1 >= _2) + _2 * (_1 < _2)

# Step 4
_1 = g(x)
_2 = np.mean(V[1,:])
V0 = _1 * (_1 >= _2) + _2 * (_1 < _2)

print("Estimator:", V0)

Estimator: 15.13836932329043


In [82]:
# Longstaff & Schwartz's algorithm

# Step 2
V2 = np.zeros(n*(m+1)).reshape((m+1),n)
V2[-1,:] = g(X[-1,:])

alpha_tau = np.zeros(l*(m+1)).reshape((m+1),l)

# Step 3
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    #alpha_tau[i,:] = minimize(lambda alpha: np.mean((V2[i+1,:] - reg(X[i,:], alpha))**2), alpha0).x
    alpha_tau[i,:] = curve_fit(reg, X[i,:], V2[i+1,:], p0=alpha0)[0]
    _1 = g(X[i,:])
    _2 = reg(X[i,:], alpha_tau[i,:])
    V2[i,:] = _1 * (_1 >= _2) + V2[(i+1),:] * (_1 < _2)

# Step 4 (not used)
_1 = g(x)
_2 = np.mean(V2[1,:])
V02 = np.mean(_1 * (_1 >= _2) + V2[1,:] * (_1 < _2))

# Step 5
B2 = rng.normal(size=m*n).reshape(m,n)
X2 = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B2]), axis=0)

V3 = np.zeros(n*(m+1)).reshape((m+1),n)
V3[-1,:] = g(X2[-1,:])

for i in range(m-1, 0, -1):
    _1 = g(X2[i,:])
    _2 = reg(X2[i,:], alpha_tau[i,:])
    V3[i,:] = _1 * (_1 >= _2) + V3[(i+1),:] * (_1 < _2)

_1 = g(x)
_2 = np.mean(V3[1,:])
V03_ = _1 * (_1 >= _2) + V3[1,:] * (_1 < _2)

V03 = np.mean(V03_)
std = np.std(V03_)
conv_interval = V03 + np.array([-1,1]) * 1.96 * std / np.sqrt(n)
final_lowerbound = conv_interval[0]

print("Estimator:", V03)
print("Standard deviation:", std / np.sqrt(n))
print("Condidence interval 95%:", conv_interval)
print("Error:", 100 * 1.96 * std / (V03 * np.sqrt(n)), "%")

Estimator: 14.696682875374023
Standard deviation: 0.22070749982301752
Condidence interval 95%: [14.26409618 15.12926958]
Error: 2.9434308634226767 %


# Martingales from Approximate Value Functions

In [83]:
# One trajectory

n2 = n

B_upperbound = rng.normal(size=m)
X_upperbound = np.cumprod(np.concatenate([[x], 1 + r * dt + sigma * np.sqrt(dt) * B_upperbound]), axis=0)
Normal = rng.normal(size=m*n2).reshape(m,n2)
M = np.zeros(m+1)
Mi = 0

for i in range(1, m):
    V_upperbound = max(g(X_upperbound[i]), reg(X_upperbound[i], alpha_tau[i,:]))
    X_next = X_upperbound[i-1] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:])
    V_Ynext = np.maximum(g(X_next), reg(X_next, alpha_tau[i,:]))
    delta = V_upperbound - np.mean(V_Ynext)
    Mi += delta
    M[i] = Mi

X_next = X_upperbound[-2] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:])
delta = g(X_upperbound[-1]) - np.mean(g(X_next))
Mi += delta
M[-1] = Mi

V0_upperbound = np.max(g(X_upperbound) - M)
print(V0_upperbound)

14.250535649337614


In [84]:
# Monte-Carlo method for n trajectories

n2 = n

Normal = rng.normal(size=m*n*n2).reshape(m,n2,n)
M = np.zeros(n*(m+1)).reshape(m+1,n)
Mi = np.zeros(n)

from tqdm import tqdm

for i in tqdm(range(1, m)):
    V_upperbound = np.maximum(g(X2[i,:]), reg(X2[i,:], alpha_tau[i,:]))
    X_next = X2[i-1,:] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:,:])
    V_Ynext = np.maximum(g(X_next), reg(X_next, alpha_tau[i,:]))
    delta = V_upperbound - np.mean(V_Ynext, axis=0)
    Mi += delta
    M[i,:] = Mi

X_next = X2[-2,:] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:,:])
delta = g(X2[-1,:]) - np.mean(g(X_next), axis=0)
Mi += delta
M[-1,:] = Mi

V0_upperbound_ = np.max(g(X2) - M, axis=0)
V0_upperbound = np.mean(V0_upperbound_)
std_upperbound = np.std(V0_upperbound_)
conv_interval_upperbound = V0_upperbound + np.array([-1,1]) * 1.96 * std_upperbound / np.sqrt(n)
final_upperbound = conv_interval_upperbound[1]

print("Estimator:", V0_upperbound)
print("Standard deviation:", std_upperbound / np.sqrt(n))
print("Condidence interval 95%:", conv_interval_upperbound)
print("Error:", 100 * 1.96 * std_upperbound / (V0_upperbound * np.sqrt(n)), "%")

100%|██████████| 9/9 [00:11<00:00,  1.29s/it]

Estimator: 15.033899131830589
Standard deviation: 0.011434225812927001
Condidence interval 95%: [15.01148805 15.05631021]
Error: 0.14907032697782943 %


In [85]:
print("Confidence interval (lower & upper bounds):", [final_lowerbound, final_upperbound])

Confidence interval (lower & upper bounds): [14.264096175720908, 15.056310214423926]


# Martingales from Stopping Rules

# Finite difference